# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel');
models = s.models;


In [15]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans BCM
BCM: SMSE 0.00148958, MSLL -3.6573, NLPD -0.8217
Processing energy_data kmeans PoE
PoE: SMSE 0.00187168, MSLL -3.2673, NLPD -0.4317


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [14]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 0.00120479, MSLL 1.1460, NLPD 3.9817


### 1.1.2 Grid q

In [13]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.00138234, MSLL 2.6867, NLPD 5.5223 - q   0.20
TERBCM: SMSE 0.00135043, MSLL 0.7086, NLPD 3.5443 - q   0.40
TERBCM: SMSE 0.00130962, MSLL 0.2198, NLPD 3.0554 - q   0.60
TERBCM: SMSE 0.00126034, MSLL 0.3913, NLPD 3.2269 - q   0.80
TERBCM: SMSE 0.00120479, MSLL 1.1460, NLPD 3.9817 - q   1.00
TERBCM: SMSE 0.00114658, MSLL 2.6566, NLPD 5.4922 - q   1.20
TERBCM: SMSE 0.00108970, MSLL 5.3330, NLPD 8.1686 - q   1.40
TERBCM: SMSE 0.00103722, MSLL 9.9481, NLPD 12.7837 - q   1.60
TERBCM: SMSE 0.00099077, MSLL 17.9091, NLPD 20.7448 - q   1.80
TERBCM: SMSE 0.00095067, MSLL 31.7833, NLPD 34.6189 - q   2.00
TERBCM: SMSE 0.00091648, MSLL 56.3025, NLPD 59.1381 - q   2.20
TERBCM: SMSE 0.00088742, MSLL 100.2918, NLPD 103.1274 - q   2.40
TERBCM: SMSE 0.00086265, MSLL 180.3968, NLPD 183.2324 - q   2.60
TERBCM: SMSE 0.00084146, MSLL 328.3394, NLPD 331.1750 - q   2.80
TERBCM: SMSE 0.00082321, MSLL 605.1418, NLPD 607.9774 - q   3.00


### 1.1.3 Find single q

In [41]:
criterion = 'TERBCM';

In [44]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.00001, lambda=10, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm = -232.20
grad_q_reg =  3664.2
ttb =  93.664
Optimizing q: remain iter - 99, q - 1.4926
grad_q_norm = -410.22
grad_q_reg =    1.3499e+04
ttb =  131.44
Optimizing q: remain iter - 98, q - 1.4651
grad_q_norm = -181.82
grad_q_reg =  1817.9
ttb =  64.145
Optimizing q: remain iter - 97, q - 1.4614
grad_q_norm = -441.06
grad_q_reg =    1.8701e+04
ttb =  152.69
Optimizing q: remain iter - 96, q - 1.4234
grad_q_norm = -298.93
grad_q_reg =    1.3976e+04
ttb =  143.25
Optimizing q: remain iter - 95, q - 1.3949
grad_q_no

In [45]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

In [47]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=1.0, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm = -146.26
grad_q_reg =  2357.8
ttb =  80.306
Optimizing q: remain iter - 99, q - 1.4549
grad_q_norm = -563.53
grad_q_reg =    2.8695e+04
ttb =  175.53
Optimizing q: remain iter - 98, q - 0.8808
grad_q_norm = -73.647
grad_q_reg =  1489.4
ttb =  108.68
Optimizing q: remain iter - 97, q - 0.8519
grad_q_norm = -63.823
grad_q_reg =  1090.6
ttb =  116.07
Optimizing q: remain iter - 96, q - 0.8309
grad_q_norm = -26.454
grad_q_reg = -374.64
ttb =  105.21
Optimizing q: remain iter - 95, q - 0.8391
grad_q_norm =  9.1359

In [9]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

In [21]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.00001, lambda=1000, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm = -468.97
grad_q_reg =  596.98
ttb =  134.77
Optimizing q: remain iter - 99, q - 1.3783
grad_q_norm = -461.10
grad_q_reg =  726.57
ttb =  170.17
Optimizing q: remain iter - 98, q - 1.2301
grad_q_norm = -151.53
grad_q_reg =  131.64
ttb =  101.76
Optimizing q: remain iter - 97, q - 1.2032
grad_q_norm = -230.65
grad_q_reg =  252.72
ttb =  119.56
Optimizing q: remain iter - 96, q - 1.1517
grad_q_norm = -271.83
grad_q_reg =  261.17
ttb =  136.48
Optimizing q: remain iter - 95, q - 1.0985
grad_q_norm = -116.09
grad_

In [22]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [10]:
criterion = 'TERBCM';

In [11]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=100.0, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  3.1559
ttb =  176.89
Optimizing qs: remain iter - 99
qs_dist =  3.0604
ttb =  141.74
Optimizing qs: remain iter - 98
qs_dist =  2.1156
ttb =  114.26
Optimizing qs: remain iter - 97
qs_dist =  2.3590
ttb =  120.80
Optimizing qs: remain iter - 96
qs_dist =  1.8144
ttb =  71.041
Optimizing qs: remain iter - 95
qs_dist =  1.9791
ttb =  118.11
Optimizing qs: remain iter - 94
qs_dist =  1.5254
ttb =  74.572
Optimizing qs: remain iter - 93
qs_dist =  0.65298
ttb =  89.267
Optimizing qs: remain iter - 92
qs_dist =  1.4532
ttb =  99.053
Optimizing qs: remain iter - 91
qs_dist =  1.7647

In [12]:
qs
save('./res/terbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.13891
   0.81935
   0.53090
   0.45057
   1.01232
   1.04775
   0.98619
   0.81528
   0.86974
   0.51547
   0.21172
   0.85112
   1.18783
   0.80023
   0.77015
   0.85269
   0.79684
   0.89355
   0.95112
   0.20975
   0.84367
   0.95760
   0.87102
   0.78369
   0.85475
   0.82499
   0.34115
   0.37391
   0.76680
   0.87368
   0.83239
   1.02595
   1.01826
   0.94088
   0.56823
   1.05961
   0.84487
   0.96488
   1.06703
   0.46111
   0.20983
   0.98621
   0.95523
   0.73318
   0.61595
   1.16551
   0.83680
   0.79389
   0.90760
   0.83194

TERBCM: SMSE 0.00126949, MSLL 1.0614, NLPD 3.8971


In [23]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.00001, lambda=1000.0, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  2.2325
ttb =  212.88
Optimizing qs: remain iter - 99
qs_dist =  0.98394
ttb =  127.28
Optimizing qs: remain iter - 98
qs_dist =  0.54428
ttb =  113.96
Optimizing qs: remain iter - 97
qs_dist =  1.1909
ttb =  129.06
Optimizing qs: remain iter - 96
qs_dist =  1.3283
ttb =  125.79
Optimizing qs: remain iter - 95
qs_dist =  0.28158
ttb =  75.071
Optimizing qs: remain iter - 94
qs_dist =  1.2744
ttb =  149.35
Optimizing qs: remain iter - 93
qs_dist =  0.50676
ttb =  93.922
Optimizing qs: remain iter - 92
qs_dist =  0.64105
ttb =  108.68
Optimizing qs: remain iter - 91
qs_dist =  0.

In [24]:
qs
save('./res/terbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.57489
   0.77089
   0.87506
   0.70509
   0.88981
   0.96903
   0.60403
   0.59968
   0.73965
   0.70464
   0.68382
   0.80556
   0.98218
   0.75338
   1.76208
   0.87039
   0.74650
   0.73683
   0.69115
   0.58947
   0.71817
   0.89328
   0.69256
   0.63153
   0.65369
   0.84869
   0.78351
   0.75597
   0.80090
   0.96977
   0.69537
   0.63133
   0.73203
   0.81988
   0.81752
   0.93146
   0.61193
   0.78944
   0.65144
   0.63898
   0.74632
   0.80671
   0.97245
   0.73268
   0.68998
   1.16702
   0.76225
   0.87421
   0.83305
   0.69373

TERBCM: SMSE 0.00126541, MSLL 0.6064, NLPD 3.4420


## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [25]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 0.00129820, MSLL -4.4819, NLPD -1.6463


### 1.2.2 Grid q

In [48]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGRBCM
TEGRBCM: SMSE 0.00205702, MSLL -4.5121, NLPD -1.6765 - q   0.20
TEGRBCM: SMSE 0.00184496, MSLL -4.5617, NLPD -1.7260 - q   0.40
TEGRBCM: SMSE 0.00159800, MSLL -4.6151, NLPD -1.7794 - q   0.60
TEGRBCM: SMSE 0.00140514, MSLL -4.6187, NLPD -1.7830 - q   0.80
TEGRBCM: SMSE 0.00129820, MSLL -4.4819, NLPD -1.6463 - q   1.00
TEGRBCM: SMSE 0.00128002, MSLL -4.0351, NLPD -1.1994 - q   1.20
TEGRBCM: SMSE 0.00133215, MSLL -2.9567, NLPD -0.1211 - q   1.40
TEGRBCM: SMSE 0.00142041, MSLL -0.6344, NLPD 2.2012 - q   1.60
TEGRBCM: SMSE 0.00150915, MSLL 4.1124, NLPD 6.9480 - q   1.80
TEGRBCM: SMSE 0.00157646, MSLL 13.5863, NLPD 16.4219 - q   2.00
TEGRBCM: SMSE 0.00161714, MSLL 32.3193, NLPD 35.1549 - q   2.20
TEGRBCM: SMSE 0.00163623, MSLL 69.2901, NLPD 72.1257 - q   2.40
TEGRBCM: SMSE 0.00164135, MSLL 142.3781, NLPD 145.2137 - q   2.60
TEGRBCM: SMSE 0.00163866, MSLL 287.3458, NLPD 290.1814 - q   2.80
TEGRBCM: SMSE 0.00163204, MSLL 576.0210, NLPD 578.8566 - q   3.00

### 1.2.3 Find single q

In [49]:
criterion = 'TEGRBCM';

In [52]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.01, lambda=1.0, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm = -0.89628
grad_q_reg =  1024.2
ttb =  20.701
Optimizing q: remain iter - 99, q - 1.2912
grad_q_norm =  4.2712
grad_q_reg =  223.27
ttb =  11.041
Optimizing q: remain iter - 98, q - 1.2447
grad_q_norm =  0.76958
grad_q_reg =  202.25
ttb =  9.9738
Optimizing q: remain iter - 97, q - 1.2033
grad_q_norm =  5.8269
grad_q_reg =  324.77
ttb =  12.931
Optimizing q: remain iter - 96, q - 1.1358
grad_q_norm = -1.4407
grad_q_reg =  139.21
ttb =  8.5186
Optimizing q: remain iter - 95, q - 1.1077
grad_q_norm =  0.33828
gr

Optimizing q: remain iter - 23, q - 0.6226
grad_q_norm =  1.0732
grad_q_reg =  12.264
ttb =  3.7975
Optimizing q: remain iter - 22, q - 0.6198
grad_q_norm =  0.26385
grad_q_reg =  9.2851
ttb =  3.7991
Optimizing q: remain iter - 21, q - 0.6179
grad_q_norm = -3.5863
grad_q_reg =  27.450
ttb =  6.0174
Optimizing q: remain iter - 20, q - 0.6130
grad_q_norm = -0.73651
grad_q_reg =  23.731
ttb =  4.8407
Optimizing q: remain iter - 19, q - 0.6083
grad_q_norm = -1.0857
grad_q_reg =  5.4135
ttb =  3.0602
Optimizing q: remain iter - 18, q - 0.6074
grad_q_norm =  0.20375
grad_q_reg =  23.878
ttb =  5.9181
Optimizing q: remain iter - 17, q - 0.6025
grad_q_norm = -0.47843
grad_q_reg =  4.2484
ttb =  2.8842
Optimizing q: remain iter - 16, q - 0.6018
grad_q_norm = -5.2205
grad_q_reg =  33.429
ttb =  7.0562
Optimizing q: remain iter - 15, q - 0.5960
grad_q_norm =  0.39939
grad_q_reg =  7.6460
ttb =  3.3763
Optimizing q: remain iter - 14, q - 0.5944
grad_q_norm = -0.47806
grad_q_reg =  2.8292
ttb =  2

In [53]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [54]:
criterion = 'TEGRBCM';

In [55]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.01, lambda=1.0, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  0.18275
ttb =  21.501
Optimizing qs: remain iter - 99
qs_dist =  0.50802
ttb =  57.742
Optimizing qs: remain iter - 98
qs_dist =  0.10266
ttb =  16.228
Optimizing qs: remain iter - 97
qs_dist =  0.29038
ttb =  25.760
Optimizing qs: remain iter - 96
qs_dist =  1.4794
ttb =  45.003
Optimizing qs: remain iter - 95
qs_dist =  0.97256
ttb =  25.527
Optimizing qs: remain iter - 94
qs_dist =  0.80909
ttb =  32.069
Optimizing qs: remain iter - 93
qs_dist =  0.27367
ttb =  15.585
Optimizing qs: remain iter - 92
qs_dist =  0.066743
ttb =  18.687
Optimizing qs: remain iter - 91
qs_dist =

In [56]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.54255
   1.39561
   1.23617
   1.28660
   1.43599
   1.59200
   1.39552
   1.54707
   1.24443
   1.24621
   1.24078
   1.48943
   1.54674
   1.53747
   1.09441
   1.45217
   1.38332
   1.27843
   1.44395
   1.43605
   1.40005
   1.47246
   1.37819
   1.47681
   1.47667
   1.12355
   1.15908
   1.38623
   1.33444
   1.46719
   1.35474
   1.28115
   1.22243
   1.28128
   1.71624
   1.83272
   1.36955
   0.95865
   1.21012
   1.43310
   1.57292
   1.38765
   1.31496
   1.52969
   1.06692
   1.29996
   1.53738
   1.20183
   1.50726
   1.34936

TEGRBCM: SMSE 0.00137250, MSLL -3.0983, NLPD -0.2627


In [57]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.01, lambda=0.100, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  0.43101
ttb =  24.765
Optimizing qs: remain iter - 99
qs_dist =  0.15959
ttb =  17.381
Optimizing qs: remain iter - 98
qs_dist =  0.13081
ttb =  30.703
Optimizing qs: remain iter - 97
qs_dist =  0.63777
ttb =  39.974
Optimizing qs: remain iter - 96
qs_dist =  0.044237
ttb =  24.612
Optimizing qs: remain iter - 95
qs_dist =  0.43948
ttb =  27.808
Optimizing qs: remain iter - 94
qs_dist =  0.017695
ttb =  13.530
Optimizing qs: remain iter - 93
qs_dist =  0.21008
ttb =  38.283
Optimizing qs: remain iter - 92
qs_dist =  0.024433
ttb =  26.215
Optimizing qs: remain iter - 91
qs_dis

In [58]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.61556
   1.10484
   1.16883
   1.32299
   1.34024
   1.79545
   1.21530
   1.88989
   1.55349
   1.43895
   1.42311
   1.42073
   1.49168
   1.19094
   1.23401
   1.15214
   1.44216
   1.22989
   1.29008
   1.67144
   1.86310
   1.22396
   1.29371
   1.33466
   1.16476
   1.05100
   1.30428
   1.55042
   1.00375
   1.66581
   1.43376
   1.82390
   0.85268
   1.69454
   1.33448
   1.83559
   1.63215
   1.26537
   1.34544
   1.83052
   1.82816
   1.29818
   1.47022
   1.29379
   0.83088
   1.38074
   1.44434
   2.02351
   1.17018
   1.75832

TEGRBCM: SMSE 0.00159285, MSLL -1.2779, NLPD 1.5577


In [39]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.0001, lambda=1000, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  1.4827
ttb =  30.566
Optimizing qs: remain iter - 99
qs_dist =  1.3919
ttb =  36.205
Optimizing qs: remain iter - 98
qs_dist =  1.4531
ttb =  24.083
Optimizing qs: remain iter - 97
qs_dist =  0.96307
ttb =  18.787
Optimizing qs: remain iter - 96
qs_dist =  1.3670
ttb =  31.292
Optimizing qs: remain iter - 95
qs_dist =  0.41863
ttb =  13.799
Optimizing qs: remain iter - 94
qs_dist =  1.3318
ttb =  28.891
Optimizing qs: remain iter - 93
qs_dist =  1.3515
ttb =  29.794
Optimizing qs: remain iter - 92
qs_dist =  0.86559
ttb =  20.933
Optimizing qs: remain iter - 91
qs_dist =  0.34

In [40]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.78746
   1.69636
   1.06446
   1.17475
   1.15575
   1.51253
   1.30678
   1.36018
   1.65369
   1.15150
   1.26463
   1.30142
   0.77491
   1.25177
   0.92270
   1.36058
   1.08058
   1.11386
   1.35962
   0.77271
   1.10309
   0.77066
   1.48699
   1.52113
   0.89852
   1.47554
   1.53344
   0.98809
   1.22791
   1.23103
   1.59881
   1.12014
   1.42073
   1.31773
   1.86944
   1.66779
   1.32987
   1.05606
   1.32831
   0.78957
   1.12896
   0.55169
   0.96630
   0.65487
   1.06784
   1.61622
   1.31413
   1.14681
   1.50208
   1.01586

TEGRBCM: SMSE 0.00147528, MSLL -3.5216, NLPD -0.6860


## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [23]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 0.00125745, MSLL 1.5450, NLPD 4.3951


### 1.3.2 Grid q

In [24]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:3.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGPoE
TEGPoE: SMSE 0.00152096, MSLL 3.5642, NLPD 6.4143 - q   0.20
TEGPoE: SMSE 0.00146682, MSLL 1.3106, NLPD 4.1606 - q   0.40
TEGPoE: SMSE 0.00140314, MSLL 0.7040, NLPD 3.5541 - q   0.60
TEGPoE: SMSE 0.00133200, MSLL 0.8148, NLPD 3.6649 - q   0.80
TEGPoE: SMSE 0.00125745, MSLL 1.5450, NLPD 4.3951 - q   1.00
TEGPoE: SMSE 0.00118446, MSLL 3.0661, NLPD 5.9162 - q   1.20
TEGPoE: SMSE 0.00111731, MSLL 5.7987, NLPD 8.6487 - q   1.40
TEGPoE: SMSE 0.00105839, MSLL 10.5390, NLPD 13.3891 - q   1.60
TEGPoE: SMSE 0.00100816, MSLL 18.7387, NLPD 21.5888 - q   1.80
TEGPoE: SMSE 0.00096588, MSLL 33.0462, NLPD 35.8963 - q   2.00
TEGPoE: SMSE 0.00093031, MSLL 58.3434, NLPD 61.1934 - q   2.20
TEGPoE: SMSE 0.00090021, MSLL 103.7325, NLPD 106.5826 - q   2.40
TEGPoE: SMSE 0.00087454, MSLL 186.3758, NLPD 189.2259 - q   2.60
TEGPoE: SMSE 0.00085243, MSLL 338.9651, NLPD 341.8151 - q   2.80
TEGPoE: SMSE 0.00083325, MSLL 624.3551, NLPD 627.2052 - q   3.00


### 1.3.3 Find single q

In [25]:
criterion = 'TEGPoE';

In [27]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=10, init_q=1.5);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Optimizing q: remain iter - 100, q - 1.5000
grad_q_norm = -270.16
grad_q_reg =  592.48
ttb =    1.2760e+04
Optimizing q: remain iter - 99, q - 1.3846
grad_q_norm = -77.959
grad_q_reg =  76.339
ttb =  4040.2
Optimizing q: remain iter - 98, q - 1.3706
grad_q_norm = -534.41
grad_q_reg =  783.84
ttb =    2.0996e+04
Optimizing q: remain iter - 97, q - 1.2215
grad_q_norm = -108.15
grad_q_reg =  136.83
ttb =  8469.1
Optimizing q: remain iter - 96, q - 1.1958
grad_q_norm = -0.75760
grad_q_reg = -2.7291
ttb =  228.99
Optimizing q: remain iter - 95, q - 1.1964
grad_q_norm = 

In [29]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [30]:
printf('Optimizing q\r\n');
iqs = 1.01 + rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=100, bs=1, lr=0.001, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Msize =

   360
   194
   469
   503
   333
   469
   571
   424
   181
   502
   497
   338
   320
   354
   277
   174
   314
   404
   340
   617
   339
   121
   411
   253
   265
   360
   287
   323
   366
   141
   510
   505
   427
   339
   292
   345
   441
   319
   436
   638
   194
   378
   378
   500
   376
   361
   185
   163
   204
   502

Optimizing qs: remain iter - 100
qs_dist =  1.5041
Optimizing qs: remain iter - 99
qs_dist =  1.7275
Optimizing qs: remain iter - 98
qs_dist =  0.74272
Optimizing qs: remain iter - 97
qs_dist =  0.47266
Optimizing qs: remain iter - 96
qs_dist =  1.1795
Optimizing qs: remain iter - 95
qs_dist =  1.0376
Optimizing qs: remain iter - 94
qs_dist =  1.0511
Optimizing qs: remain iter - 93
qs_dist =  0.60297
Optimizing qs: remain iter - 92
qs_dist =  1.1140
Optimizing qs: remain iter - 91
qs_dist =  0.66522
Optimizing qs: remain iter - 90
qs_dist =  0.099498
Optimizing qs: remain iter - 89
qs_dist =  0.26207
Optimizing qs: rema

In [31]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.58045
   0.84382
   0.91310
   0.77310
   0.74332
   0.88934
   0.73395
   0.71239
   0.69959
   0.76793
   0.80011
   0.79673
   1.03848
   0.79762
   1.76134
   0.89089
   0.76726
   0.79258
   0.79486
   0.65253
   0.73700
   0.85995
   0.66722
   0.69396
   0.75737
   0.97535
   0.74619
   0.81695
   0.86614
   1.03477
   0.73302
   0.72435
   0.86696
   0.89470
   0.88851
   0.96970
   0.72331
   0.78168
   0.71919
   0.66636
   0.85685
   0.73035
   0.98696
   0.81426
   0.84020
   0.92244
   0.77273
   0.92182
   0.95294
   0.75760

TEGPoE: SMSE 0.00132153, MSLL 0.9445, NLPD 3.7946
